In [ ]:
!pip install protobuf==3.20.0

In [1]:
import sys
sys.path.append("/ibkr")

import os
import datetime
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import pytz


import passwords
from databaseClass import DB
from utils import generate_df_from_sql_file, generate_list_from_sql_file
from utils import DataProcessor
from utils import ModelBuilder
from utils import Predictor
pd.set_option('display.max_rows', None)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Restrict TensorFlow to only use the first GPU
    tf.config.set_visible_devices(gpus[0], 'GPU')

    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
!nvidia-smi

In [ ]:
# !pip install scikit-learn

In [ ]:
userName = passwords.userName
userPass = passwords.userPass
dataBaseName = passwords.dataBaseName
host = passwords.host

In [ ]:
db = DB(userName=userName, userPass=userPass, dataBaseName=dataBaseName, host='ibkr_db', docker=True)

In [ ]:
db.tables

In [ ]:
input_sql_file='../sql_files/test.sql'
df = generate_df_from_sql_file(input_sql_file, db)

In [ ]:
df.head(10)

In [ ]:
df = df.iloc[1:]

In [ ]:
def process_df(df):
    # Set 'timestamp' as the index
    df = df.set_index('timestamp')
    
    # Pivot the table and also include 'volume' where ticktype is 5
    df_pivot = df.pivot_table(index=df.index, columns='ticktype', values=['price', 'volume'])
    df_pivot.columns = ['_'.join(map(str,i)) for i in df_pivot.columns]
    
    # Resample the data per second and fill forward any NaN values
    df_resampled = df_pivot.resample('1S').agg({'price_1': 'last', 'price_2': 'last', 'price_4': 'last', 'volume_5': 'sum'}).ffill()
    
    #reshape
    df_resampled['Open'] = df_resampled['price_1']
    df_resampled['High'] = df_resampled[['price_1', 'price_2', 'price_4']].max(axis=1)
    df_resampled['Low'] = df_resampled[['price_1', 'price_2', 'price_4']].min(axis=1)
    df_resampled['Close'] = df_resampled['price_4']
    df_resampled['Volume'] = df_resampled['volume_5']
    df_resampled = df_resampled[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    return df_resampled

start_time = time.time()
processed_df = process_df(df)
end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

def scale_shift_data(df, look_ahead):
    # Normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(df)

    # Convert scaled array into dataframe
    df_scaled = pd.DataFrame(scaled, index=df.index, columns=df.columns)

    # Shift the dataframe to create the labels
    df_scaled_shifted = df_scaled.shift(-look_ahead)

    # Drop the last 'look_ahead' rows
    df_scaled = df_scaled.iloc[:-look_ahead]
    df_scaled_shifted = df_scaled_shifted.iloc[:-look_ahead]

    return df_scaled, df_scaled_shifted['Close'], scaler

# Preparing data for 5 second prediction
X, y, scaler_5s = scale_shift_data(processed_df, look_ahead=5)

# Preparing data for 1 minute prediction
X, y, scaler_1m = scale_shift_data(processed_df, look_ahead=60)

def create_dataset(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 60

# Reshape to [samples, time_steps, n_features]
X_5s, y_5s = create_dataset(X, y, TIME_STEPS)
X_1m, y_1m = create_dataset(X, y, TIME_STEPS)

train_size = int(len(X_5s) * 0.8)
test_size = len(X_5s) - train_size

X_train_5s, X_test_5s = X_5s[0:train_size], X_5s[train_size:len(X_5s)]
y_train_5s, y_test_5s = y_5s[0:train_size], y_5s[train_size:len(y_5s)]

X_train_1m, X_test_1m = X_1m[0:train_size], X_1m[train_size:len(X_1m)]
y_train_1m, y_test_1m = y_1m[0:train_size], y_1m[train_size:len(y_1m)]

In [ ]:
X_train_5s

In [ ]:
# Train the Model

# Number of features in the data. In your case, it's 4 (price_1, price_2, price_4, volume_5)
n_features = 5 

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(TIME_STEPS, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_5s, y_train_5s, epochs=20, batch_size=64, validation_data=(X_test_5s, y_test_5s), shuffle=False)

In [ ]:
# You can plot the loss with:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
model.save('../models/model.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
# returns a compiled model identical to the previous one
model = load_model('../models/model.h5')

In [ ]:
def main_build_model(df, look_ahead=5):
    
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC') #adding this to update to utc

    # Create a DataProcessor instance
    processor = DataProcessor(df) 
    # Process the df
    processor.process_df()
    
    # Scale and shift the data
    scaler, close_scaler = processor.scale_shift_data(look_ahead)
    # Create the X and y datasets
    X, y = processor.scaled_df, processor.shifted_df['Close']
    # Create train test split
    X_train, X_test, y_train, y_test = processor.create_train_test_split(X, y)
    # Create final train and test datasets
    TIME_STEPS = 60
    X_train, y_train = processor.create_dataset(X_train, y_train, TIME_STEPS)
    X_test, y_test = processor.create_dataset(X_test, y_test, TIME_STEPS)
    # Number of features in the data
    n_features = X_train.shape[2]
    
    # Create a ModelBuilder instance and build the model
    builder = ModelBuilder(n_features, TIME_STEPS)
    # Train the model
    model, history = builder.train_model(X_train, y_train, X_test, y_test, epochs=20, batch_size=64)
    # Save the model
    model_path = '../models/model.h5'
    builder.save_model(model_path)
    # Plot loss
    builder.plot_loss(history)
    # Create a Predictor instance
    predictor = Predictor(model, processor)
    # Predict
    predictions = predictor.predict(look_ahead, TIME_STEPS)
    # Rescale predictions
    rescaled_predictions = predictor.rescale_prediction(predictions)
    return rescaled_predictions

In [ ]:
main_build_model(df, 5)

In [ ]:
df.head()